### Prepare Repo

In [1]:
# !git clone https://github.com/facebookresearch/av_hubert.git

# %cd av_hubert
# !git submodule init
# !git submodule update
# !pip install scipy
# !pip install sentencepiece
# !pip install python_speech_features
# !pip install scikit-video

# %cd fairseq
# !python -m pip install ./

In [2]:
%cd E:/university/FYT/repos/multi_modal_ser/finetune_encoder/audio_video/av_hubert/avhubert

import sys
sys.path.append("E:/university/FYT/repos/multi_modal_ser/finetune_encoder/audio_video/av_hubert/fairseq")
from fairseq import checkpoint_utils, options, tasks, utils
import cv2
import tempfile
import torch
from torch.nn.utils.rnn import pad_sequence
from transformers import Trainer, TrainingArguments
import utils as avhubert_utils
from argparse import Namespace
from IPython.display import HTML
from scipy.io import wavfile
from python_speech_features import logfbank
import numpy as np
import sys
print(sys.version)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import torch.nn as nn

E:\university\FYT\repos\multi_modal_ser\finetune_encoder\audio_video\av_hubert\avhubert


D:\anaconda\envs\py37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


3.7.1 (default, Oct 28 2018, 08:39:03) [MSC v.1912 64 bit (AMD64)]


In [3]:
def stacker(feats, stack_order):
    """
    Concatenating consecutive audio frames
    Args:
    feats - numpy.ndarray of shape [T, F]
    stack_order - int (number of neighboring frames to concatenate
    Returns:
    feats - numpy.ndarray of shape [T', F']
    """
    feat_dim = feats.shape[1]
    if len(feats) % stack_order != 0:
        res = stack_order - len(feats) % stack_order
        res = np.zeros([res, feat_dim]).astype(feats.dtype)
        feats = np.concatenate([feats, res], axis=0)
    feats = feats.reshape((-1, stack_order, feat_dim)).reshape(-1, stack_order*feat_dim)
    return feats

### Download Model

In [4]:
# !wget https://dl.fbaipublicfiles.com/avhubert/model/lrs3_vox/vsr/base_vox_433h.pt -O E:/check_pts/avhubert.pt

### Build Model Pipeline

In [5]:
user_dir = "E:/university/FYT/repos/multi_modal_ser/finetune_encoder/audio_video/av_hubert/avhubert"
utils.import_user_module(Namespace(user_dir=user_dir))
ckpt_path = "E:/check_pts/avhubert.pt"
models, saved_cfg, task = checkpoint_utils.load_model_ensemble_and_task([ckpt_path])  
model = models[0]
if hasattr(models[0], 'decoder'):
    print(f"Checkpoint: fine-tuned")
    model = models[0].encoder.w2v_model
else:
    print(f"Checkpoint: pre-trained w/o fine-tuning")

Checkpoint: fine-tuned


In [6]:
smp_id = "Ses01F_impro05_M017"
video_path = "E:/datasets/preprocessed/face_raw/{}/{}.mp4".format(smp_id.split("_")[0][:-1],smp_id)
audio_path = "E:/datasets/preprocessed/spectrogram/raw/{}.wav".format(smp_id)

# Load Video
# Mute transform first
transform = avhubert_utils.Compose([
  avhubert_utils.Normalize(0.0, 255.0),
  avhubert_utils.CenterCrop((task.cfg.image_crop_size, task.cfg.image_crop_size)),
  avhubert_utils.Normalize(task.cfg.image_mean, task.cfg.image_std)])

frames = avhubert_utils.load_video(video_path)
# frames = transform(frames)
frames = torch.FloatTensor(frames).unsqueeze(dim=0).unsqueeze(dim=0)
video_feats = frames

# Load Audio
sample_rate, wav_data = wavfile.read(audio_path)
in_features = model.feature_extractor_audio.proj.in_features
assert sample_rate == 16_000 and len(wav_data.shape) == 1
audio_feats = logfbank(wav_data, samplerate=sample_rate).astype(np.float32) # [T, F]
audio_feats = stacker(audio_feats, in_features//26) # [T/stack_order_audio, F*stack_order_audio]

# Match Audio Video
print(audio_feats.shape)
print(video_feats.shape)

diff = audio_feats.shape[0] - video_feats.shape[2]
if diff < 0:
    audio_feats = np.concatenate([audio_feats, np.zeros([-diff, audio_feats.shape[-1]], dtype=audio_feats.dtype)])
elif diff > 0:
    audio_feats = audio_feats[:-diff]

import torch.nn.functional as F
audio_feats = torch.from_numpy(audio_feats.astype(np.float32)).T
audio_feats = F.layer_norm(audio_feats, audio_feats.shape[1:])
audio_feats = audio_feats.unsqueeze(dim=0)


(160, 104)
torch.Size([1, 1, 192, 88, 88])


In [7]:
model.eval()
with torch.no_grad():
    # Specify output_layer if you want to extract feature of an intermediate layer
    feature, _ = model.extract_finetune(source={'video': video_feats , 
                                                'audio': audio_feats}, padding_mask=None, output_layer=None)
    feature = feature.squeeze(dim=0) 
print(f"Video feature shape: {feature.shape}")

Video feature shape: torch.Size([192, 768])


In [8]:
### Save Model Standalone Checkpoints

In [9]:
from torch.utils.data import Dataset, Subset
import os
AUDIORATE=16000
class MMSERDataset(Dataset):
    """multi model ser dataset."""
    
    def stacker(self, feats, stack_order):
        """
        Concatenating consecutive audio frames
        Args:
        feats - numpy.ndarray of shape [T, F]
        stack_order - int (number of neighboring frames to concatenate
        Returns:
        feats - numpy.ndarray of shape [T', F']
        """
        feat_dim = feats.shape[1]
        if len(feats) % stack_order != 0:
            res = stack_order - len(feats) % stack_order
            res = np.zeros([res, feat_dim]).astype(feats.dtype)
            feats = np.concatenate([feats, res], axis=0)
        feats = feats.reshape((-1, stack_order, feat_dim)).reshape(-1, stack_order*feat_dim)
        return feats
        
    def __load_label__(self, cutmap_path):
        sheet_df = pd.DataFrame()
        for ses in range(1, 6):
            extractionmapPATH = cutmap_path + \
                str(ses)+'.xlsx'
            xl = pd.ExcelFile(extractionmapPATH)
            sheets = xl.sheet_names
            for sheet in sheets:
                sheet_df = pd.concat([sheet_df, xl.parse(sheet)])
        self.df_ = sheet_df
        
        # remove samples not agreed
        self.df_ = pd.merge(self.df_, self.df_text, on=["smp_id"])
        self.df_["emotion_id"] = self.df_["emotion"].map(self.emo2id)
        self.df_ = self.df_[self.df_["emotion_id"].notna()].reset_index(drop=True)
        self.df_["session"] = self.df_["smp_id"].apply(lambda x: x.split("_")[0])
        self.df_ = self.df_[self.df_["smp_id"].str.startswith("Ses01F_impro")].reset_index(drop=True)
        
    def __load_text__(self, text_path):
        self.df_text = pd.read_csv(text_path)
        pass
    
    def __load_audio__(self, fn_path):
        self.fn_list = list(self.df_["smp_id"])
        self.raw_list = []
        for fn in self.fn_list:
            self.raw_list.append(wavfile.read(os.path.join(fn_path, fn)+'.wav')[1])
    
    def __load_video__(self, idx):
        frames = avhubert_utils.load_video(os.path.join(self.video_path, idx.split("_")[0][:-1], idx+".mp4"))
#         transform = avhubert_utils.Compose([
#           avhubert_utils.Normalize(0.0, 255.0),
#           avhubert_utils.CenterCrop((task.cfg.image_crop_size, task.cfg.image_crop_size)),
#           avhubert_utils.Normalize(task.cfg.image_mean, task.cfg.image_std)])
        # frames = transform(frames)
        frames = torch.FloatTensor(frames).unsqueeze(dim=0).unsqueeze(dim=0)
        video_feats = frames
        return video_feats
    
    def __init__(self, 
                 fn_path, 
                 cutmap_path, 
                 text_path, 
                 video_path, 
                 emo2id,
                 audio_in_features = 104):
        
        self.emo2id = emo2id
        self.audio_in_features = audio_in_features
        self.video_path = video_path
        self.__load_text__(text_path)
        self.__load_label__(cutmap_path)
        self.__load_audio__(fn_path)
        
    def __len__(self):
        return self.df_.shape[0]
    
    def __getsingle__(self, idx):
        raw_audio = self.raw_list[idx]
        video_feats = self.__load_video__(self.fn_list[idx])
        audio_feats = logfbank(raw_audio, samplerate=AUDIORATE).astype(np.float32) # [T, F]
        audio_feats = self.stacker(audio_feats, self.audio_in_features//26) # [T/stack_order_audio, F*stack_order_audio]

        diff = audio_feats.shape[0] - video_feats.shape[2]
        if diff < 0:
            audio_feats = np.concatenate([audio_feats, np.zeros([-diff, audio_feats.shape[-1]], dtype=audio_feats.dtype)])
        elif diff > 0:
            audio_feats = audio_feats[:-diff]

        with torch.no_grad():
            audio_feats = torch.from_numpy(audio_feats.astype(np.float32)).T
            audio_feats = F.layer_norm(audio_feats, audio_feats.shape[1:])
            audio_feats = audio_feats.unsqueeze(dim=0)
        return audio_feats, video_feats
    
    def collate(self, audio, video, max_size=500):
        padded_audio = pad_sequence([a.T.squeeze() for a in audio]+[torch.empty(500, 104)], batch_first=True)[:-1]
#         print(padded_audio)
        padded_video = pad_sequence([v.squeeze() for v in video]+[torch.empty(500,88,88)], batch_first=True)[:-1, np.newaxis, : ,:,:]
        mask = torch.zeros_like(padded_audio)
        mask[padded_audio != 0] = 1
        return padded_audio, padded_video, mask
        
    def __getitem__(self, idx):
        if isinstance(idx, slice):
            ret_dict = {}
            for key in self.__getitem__(0).keys():
                ret_dict[key] = [self.__getitem__(i)[key] for i in range(*idx.indices(len(self)))]
            
#             padded_audio, padded_video, padding_mask = self.collate(ret_dict["audio"], ret_dict["video"])
#             ret_dict["padding_mask"] = padding_mask
#             ret_dict["audio"] = padded_audio.transpose(1, 2)
#             ret_dict["video"] = padded_video

            ret_dict["audio"] = torch.stack(ret_dict["audio"])
            ret_dict["video"] = torch.stack(ret_dict["video"])
            ret_dict["padding_mask"] = torch.stack(ret_dict["padding_mask"])
            return ret_dict
        
        else:
            audio_feats, video_feats = self.__getsingle__(idx)
            padded_audio, padded_video, padding_mask = self.collate([audio_feats], [video_feats])
            
            return {
                "padding_mask": padding_mask[0][:500, :],
                "sess": list(self.df_["session"])[idx],
                "fn": self.fn_list[idx],
                "audio": padded_audio[0][:500, :].transpose(0,1),
                "video": padded_video[0][:, :500, :, :],
                "text": list(self.df_["transcript"])[idx],
                "labels": list(self.df_["emotion_id"])[idx]
            }

In [10]:
mmser_ds = torch.load("E:/datasets/preprocessed/dataset/avhubert_ds.pt")

In [11]:
print(mmser_ds[2:4]['audio'].shape)
print(mmser_ds[2:4]['video'].shape)
print(mmser_ds[2:4]['padding_mask'].shape)

D:\anaconda\envs\py37\lib\site-packages\ipykernel_launcher.py:100: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3281.)


torch.Size([2, 104, 500])
torch.Size([2, 1, 500, 88, 88])
torch.Size([2, 500, 104])


In [12]:
print(mmser_ds[24]['audio'].shape)
print(mmser_ds[24]['video'].shape)
print(mmser_ds[24]['padding_mask'].shape)

torch.Size([104, 500])
torch.Size([1, 500, 88, 88])
torch.Size([500, 104])


In [13]:
import os
AUDIORATE = 16000

outputs = model.extract_finetune(mmser_ds[:2])
outputs

(tensor([[[-0.2509, -0.0123, -0.0282,  ...,  0.0534,  0.1481, -0.1306],
          [-0.0297, -0.0215, -0.0957,  ..., -0.1796,  0.2736, -0.0539],
          [ 0.1570, -0.0434, -0.1272,  ..., -0.3133,  0.2696,  0.0819],
          ...,
          [ 0.0884,  0.1770,  0.0067,  ..., -0.2538,  0.0719,  0.0278],
          [ 0.0315,  0.1851,  0.0935,  ..., -0.2343,  0.1228, -0.0029],
          [-0.1848,  0.1584,  0.2137,  ..., -0.1957,  0.1717, -0.0370]],
 
         [[-0.2540, -0.0186, -0.0562,  ...,  0.0506,  0.1489, -0.1205],
          [-0.0943, -0.0356, -0.0993,  ..., -0.1062,  0.2671, -0.1125],
          [ 0.1451, -0.0354, -0.1455,  ..., -0.3019,  0.2774,  0.0684],
          ...,
          [ 0.0968,  0.1664, -0.0320,  ..., -0.2547,  0.0713,  0.0353],
          [ 0.0472,  0.1763,  0.0603,  ..., -0.2347,  0.1307,  0.0017],
          [-0.1711,  0.1546,  0.1980,  ..., -0.1913,  0.1879, -0.0391]]],
        grad_fn=<NativeLayerNormBackward0>),
 None)

In [14]:
outputs[0].shape

torch.Size([2, 500, 768])

In [15]:
attention_mask = mmser_ds[:4]["padding_mask"]

In [16]:
def forward_padding_mask(
        features: torch.Tensor, padding_mask: torch.Tensor,
    ):
        extra = padding_mask.size(1) % features.size(1)
        if extra > 0:
            padding_mask = padding_mask[:, :-extra]
        padding_mask = padding_mask.view(
            padding_mask.size(0), features.size(1), -1
        )
        padding_mask = padding_mask.all(-1)
        return padding_mask

In [17]:
forwarded_attention_mask = forward_padding_mask(outputs[0], attention_mask)

### Define the model

In [18]:
class AVHUBERTClassifier(torch.nn.Module):

    def __init__(self, encoder, hidden_size, proj_size, num_labels=4):
        super(AVHUBERTClassifier, self).__init__()
        self.encoder = encoder
        self.num_labels = num_labels
        self.hidden_size = hidden_size
        self.proj_size = proj_size
        self.num_labels = num_labels
        
        self.projector = nn.Linear(self.hidden_size, self.proj_size)
        self.classifier = nn.Linear(self.proj_size, num_labels)

    def forward(
        self,
        audio, video, padding_mask, **kwargs
    ):
        with torch.no_grad():
            outputs = self.encoder.extract_finetune(
                {
                    "audio": audio, 
                    "video": video,
                    "padding_mask": padding_mask
                }
            )
            hidden_states = outputs[0]
        
        hidden_states = self.projector(hidden_states)
        attention_mask = padding_mask
        padding_mask = forward_padding_mask(hidden_states, attention_mask)
        hidden_states[~padding_mask] = 0.0
        pooled_output = hidden_states.sum(dim=1) / padding_mask.sum(dim=1).view(-1, 1)
        logits = self.classifier(pooled_output)
        return {"logits":logits}
    

In [19]:
classifier = AVHUBERTClassifier(model, 768, 256, mmser_ds.df_["emotion_id"].nunique())
classifier(**mmser_ds[:4])

{'logits': tensor([[ 0.0574,  0.0383, -0.0327, -0.0119],
         [ 0.0603,  0.0359, -0.0339, -0.0071],
         [ 0.0555,  0.0390, -0.0274, -0.0182],
         [ 0.0460,  0.0393, -0.0263, -0.0172]], grad_fn=<AddmmBackward0>)}

In [20]:
def build_ds(sess_id):
    train_size = int(len(sess_ds[sess_id+"_train"])*0.75)
    val_size = len(sess_ds[sess_id+"_train"])-train_size
    train_set, val_set = torch.utils.data.random_split(sess_ds[sess_id+"_train"], [train_size, val_size])
    test_set = sess_ds[sess_id+"_test"]

    print("Train Samples:", len(train_set))
    print("Val Samples:", len(val_set))
    print("Test Samples:", len(test_set))
    
    return train_set, val_set, test_set

In [21]:
meta_df_ = mmser_ds.df_
sess_dict = meta_df_.groupby("session").groups
all_indices = set(meta_df_.index.tolist())
sess_ds = {}
for sess in sess_dict:
#     sess_ds[sess+"_train"] = Subset(mmser_ds, 
#                                     indices=list(all_indices-set(sess_dict[sess])))
    sess_ds[sess+"_test"] = Subset(mmser_ds, 
                                    indices=sess_dict[sess])
    sess_ds[sess+"_train"] = Subset(mmser_ds, 
                                    indices=sess_dict[sess])
    
    
    
SESS_ID = list(sess_dict.keys())[0]
train_set, val_set, test_set = build_ds(SESS_ID)

Train Samples: 180
Val Samples: 61
Test Samples: 241


In [22]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels").type(torch.LongTensor).to(device)
        inputs = {"video": inputs["video"], "audio":inputs["audio"], "padding_mask":inputs["padding_mask"]}
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = nn.CrossEntropyLoss() 
        loss = loss_fct(logits.view(-1, self.model.num_labels), labels.view(-1))        
        return (loss, outputs) if return_outputs else loss
    
def weighted_acc(y_true, y_pred):
    return np.sum((np.array(y_pred).ravel() == np.array(y_true).ravel()))*1.0/len(y_true)
    
def unweighted_acc(y_true, y_pred):
    y_true = np.array(y_true).ravel()
    y_pred = np.array(y_pred).ravel()
    classes = np.unique(y_true)
    classes_accuracies = np.zeros(classes.shape[0])
    for num, cls in enumerate(classes):
        classes_accuracies[num] = weighted_acc(y_true[y_true == cls], y_pred[y_true == cls])
    return np.mean(classes_accuracies)

def compute_metrics(eval_preds):
    logits, labels = eval_preds.predictions, eval_preds.label_ids
    predictions = np.argmax(logits, axis=-1)

    metric_f1 = load_metric("f1")
    metric_acc = load_metric("accuracy")
    logits, labels = eval_preds.predictions, eval_preds.label_ids
    predictions = np.argmax(logits, axis=-1)
    f1_ = metric_f1.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    acc_ = metric_acc.compute(predictions=predictions, references=labels)["accuracy"]
    
    return {"wa":weighted_acc(labels, predictions), 
            "ua":unweighted_acc(labels, predictions),
            "f1":f1_, 
            "accuracy":acc_}

In [23]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_set, batch_size=2)
# next(iter(train_loader))

In [24]:
output_dir=os.path.join("avhubert")

training_args = TrainingArguments(output_dir,report_to="wandb")
training_args.remove_unused_columns=False
training_args.per_device_train_batch_size=4
training_args.per_device_eval_batch_size=4
training_args.logging_steps = 3 # int(1000/training_args.per_device_train_batch_size)
training_args.eval_steps = 3 #int(1000/training_args.per_device_train_batch_size)
training_args.evaluation_strategy="steps" 
training_args.logging_strategy="steps"
training_args.load_best_model_at_end=True,
training_args.save_strategy = "no"
training_args.learning_rate=1e-3
training_args.num_train_epochs=50

trainer = CustomTrainer(
    model=classifier,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=val_set,
    compute_metrics=compute_metrics,
)

In [25]:
trainer.train()

D:\anaconda\envs\py37\lib\site-packages\transformers\optimization.py:415: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: brucehu. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
3,1.394400,No log
6,1.294600,No log
9,1.196500,No log


KeyboardInterrupt: 